## softmax

In [12]:
import numpy as np
import random

In [13]:
def softmax(x):
											#x=np.array([[1001,1002],[3,4]])
	if len(x.shape) > 1:					#x.shape=(2, 2)  len(x.shape)=2				
		tmp = np.max(x, axis = 1)			#np.max(x, axis = 1)=array([1002,  4])， max in each row
		x -= tmp.reshape((x.shape[0], 1))	#tmp.reshape((x.shape[0], 1))， tmp becomes 2row1column
		x = np.exp(x)						#xi - max this row, then exp
		tmp = np.sum(x, axis = 1)			#array([ 1.36787944,  1.36787944])，sum of each row
		x /= tmp.reshape((x.shape[0], 1))	#xi / sum this row
	
	else:									#x=[1,2]   x.shape=(2,)   len(x.shape)=1
		tmp = np.max(x)
		x -= tmp
		x = np.exp(x)
		tmp = np.sum(x)
		x /= tmp
	
	return x

## gradcheck

In [14]:
# Function: 
# for each element in x
# compare derivative calculated by formular and calculus
# f: 1st parameter is cost function, 2nd parameter is gradient
def gradcheck_naive(f, x):
	
	#Return an object capturing the current internal state of the generator
	rndstate = random.getstate()			#why use state??????
	random.setstate(rndstate)
	fx, grad = f(x)							#fx=np.sum(x ** 2), grad=x * 2 
	h = 1e-4
	
	#Efficient multi-dimensional iterator object to iterate over arrays
	# Iterate over all indexes in x
	it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])	
	
	while not it.finished:
		ix = it.multi_index					#starts from (0, 0) then (0, 1)
		
		x[ix] += h							#To calculate [f(xi+h)-f(xi-h)] / 2h
		random.setstate(rndstate)
		fxh, _ = f(x)
		x[ix] -= 2*h
		random.setstate(rndstate)
		fxnh, _ = f(x)
		x[ix] += h
		numgrad = (fxh - fxnh) / 2 / h
											#To compare gradient calculated by formular and calculus
		reldiff = abs(numgrad - grad[ix]) / max(1, abs(numgrad), abs(grad[ix]))
		if reldiff > 1e-5:
			print "Gradient check failed."
			print "First gradient error found at index %s" % str(ix)
			print "Your gradient: %f \t Numerical gradient: %f" % (grad[ix], numgrad)
			return
		
		it.iternext()
		
	print "Gradient check passed"

## word2vec

In [16]:
def normalizeRows(x):

	N = x.shape[0]
	x /= np.sqrt(np.sum(x**2, axis=1)).reshape((N,1)) + 1e-30
	
	return x


def test_normalize_rows():
	print "Testing normalizeRows..."
	x = normalizeRows(np.array([[3.0, 4.0],[1, 2]]))
	print x
	assert (np.amax(np.fabs(x - np.array([[0.6,0.8],[0.4472136,0.89442719]]))) <= 1e-6)
	print ""

### """ Softmax cost function for word2vec models """

In [17]:
def softmaxCostAndGradient(predicted, target, outputVectors, dataset):
	""" Softmax cost function for word2vec models """
	
	probabilities = softmax(predicted.dot(outputVectors.T))			
	cost = -np.log(probabilities[target])
	
	delta = probabilities
	delta[target] -= 1
	
	N = delta.shape[0]												#delta.shape = (5,)
	D = predicted.shape[0]											#predicted.shape = (3,)
	grad = delta.reshape((N, 1)) * predicted.reshape((1, D))
	gradPred = (delta.reshape((1, N)).dot(outputVectors)).flatten()
	
	return cost, gradPred, grad

### """ Skip-gram model in word2vec """

In [18]:
def skipgram(currentWord, C, contextWords, tokens, inputVectors, outputVectors,
	dataset, word2vecCostAndGradient = softmaxCostAndGradient):
	""" Skip-gram model in word2vec """
	
	currentI = tokens[currentWord]						#the order of this center word in the whole vocabulary
	predicted = inputVectors[currentI, :]				#turn this word to vector representation
	
	cost = 0.0
	gradIn = np.zeros(inputVectors.shape)
	gradOut = np.zeros(outputVectors.shape)
	for cwd in contextWords:							#contextWords is of 2C length
		idx = tokens[cwd]
		cc, gp, gg = word2vecCostAndGradient(predicted, idx, outputVectors, dataset)
		cost += cc										#final cost/gradient is the 'sum' of result calculated by each word in context
		gradOut += gg
		gradIn[currentI, :] += gp
	
	return cost, gradIn, gradOut

###  word2vec_sgd_wrapper

In [19]:
def word2vec_sgd_wrapper(word2vecModel, tokens, wordVectors, dataset, C, word2vecCostAndGradient = softmaxCostAndGradient):
	batchsize = 50
	cost = 0.0
	grad = np.zeros(wordVectors.shape)   #each element in wordVectors has a gradient
	N = wordVectors.shape[0]
	inputVectors = wordVectors[:N/2, :]
	outputVectors = wordVectors[N/2:, :]
	for i in xrange(batchsize):									#train word2vecModel for 50 times
		C1 = random.randint(1, C)
		centerword, context = dataset.getRandomContext(C1)		#randomly choose 1 word, and generate a context of it
		
		if word2vecModel == skipgram:
			denom = 1
		else:
			denom = 1
		
		c, gin, gout = word2vecModel(centerword, C1, context, tokens, inputVectors, outputVectors, dataset, word2vecCostAndGradient)
		cost += c / batchsize / denom							#calculate the average
		grad[:N/2, :] += gin / batchsize / denom
		grad[N/2:, :] += gout / batchsize / denom
	
	return cost, grad

## test_word2vec

In [20]:
def test_word2vec():
    # Interface to the dataset for negative sampling
    dataset = type('dummy', (), {})()
    def dummySampleTokenIdx():
        return random.randint(0, 4)

    def getRandomContext(C):
        tokens = ["a", "b", "c", "d", "e"]
        return tokens[random.randint(0,4)], [tokens[random.randint(0,4)] \
           for i in xrange(2*C)]
    dataset.sampleTokenIdx = dummySampleTokenIdx
    dataset.getRandomContext = getRandomContext

    random.seed(31415)
    np.random.seed(9265)
    dummy_vectors = normalizeRows(np.random.randn(10,3))
    dummy_tokens = dict([("a",0), ("b",1), ("c",2),("d",3),("e",4)])

    print "==== Gradient check for skip-gram ===="
    gradcheck_naive(lambda vec: word2vec_sgd_wrapper(skipgram, dummy_tokens, vec, dataset, 5), dummy_vectors)

    print "\n=== Results ==="
    print skipgram("c", 3, ["a", "b", "e", "d", "b", "c"], dummy_tokens, dummy_vectors[:5,:], dummy_vectors[5:,:], dataset)

if __name__ == "__main__":
    test_word2vec()

==== Gradient check for skip-gram ====
Gradient check passed

=== Results ===
(11.166109001533981, array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [-1.26947339, -1.36873189,  2.45158957],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]]), array([[-0.41045956,  0.18834851,  1.43272264],
       [ 0.38202831, -0.17530219, -1.33348241],
       [ 0.07009355, -0.03216399, -0.24466386],
       [ 0.09472154, -0.04346509, -0.33062865],
       [-0.13638384,  0.06258276,  0.47605228]]))


# '''Run word2vec'''

## SGD

In [1]:
SAVE_PARAMS_EVERY = 1000

import glob
import random
import numpy as np
import os.path as op
import cPickle as pickle

def load_saved_params():
	""" A helper function that loads previously saved parameters and resets iteration start """
	st = 0
	for f in glob.glob("saved_params_*.npy"):
		iter = int(op.splitext(op.basename(f))[0].split("_")[2])
		if (iter > st):
			st = iter
	
	if st > 0:
		with open("saved_params_%d.npy" % st, "r") as f:
			params = pickle.load(f)
			state = pickle.load(f)
		return st, params, state
	else:
		return st, None, None

def save_params(iter, params):
	with open("saved_params_%d.npy" % iter, "w") as f:
		pickle.dump(params, f)
		pickle.dumpy(random.getstate(), f)

def sgd(f, x0, step, iterations, postprocessing = None, useSaved = False, PRINT_EVERY = 10):
	""" Stochastic Gradient Descent """
	ANNEAL_EVERY = 20000
	
	if useSaved:
		start_iter, oldx, state = load_saved_params()
		if start_iter > 0:
			x0 = oldx;
			step *= 0.5 ** (start_iter / ANNEAL_EVERY)
		
		if state:
			random.setstate(state)
	
	else:
		start_iter = 0
	
	x = x0
	
	if not postprocessing:
		postprocessing = lambda x: x
	
	expcost = None
	
	for iter in xrange(start_iter + 1, iterations + 1):
		
		cost = None
		
		cost, grad = f(x)
		x -= step * grad
		
		x = postprocessing(x)
		
		if iter % PRINT_EVERY == 0:
			if not expcost:
				expcost = cost
			else:
				expcost = .95 * expcost + .05 * cost
			print "iter %d: %f" % (iter, expcost)
			
		if iter % SAVE_PARAMS_EVERY == 0 and useSaved:
			save_params(iter, x)
		
		if iter % ANNEAL_EVERY == 0:
			step *= 0.5
		
	return x

### cs224.data_utils.py

In [ ]:
import cPickle as pickle
import numpy as np
import os
import random

class StanfordSentiment:
    def __init__(self, path=None, tablesize = 1000000):
        if not path:
            path = "cs224d/datasets/stanfordSentimentTreebank"

        self.path = path
        self.tablesize = tablesize

    def tokens(self):
        if hasattr(self, "_tokens") and self._tokens:
            return self._tokens

        tokens = dict()
        tokenfreq = dict()
        wordcount = 0
        revtokens = []
        idx = 0

        for sentence in self.sentences():
            for w in sentence:
                wordcount += 1
                if not w in tokens:
                    tokens[w] = idx
                    revtokens += [w]
                    tokenfreq[w] = 1
                    idx += 1
                else:
                    tokenfreq[w] += 1

        tokens["UNK"] = idx
        revtokens += ["UNK"]
        tokenfreq["UNK"] = 1
        wordcount += 1

        self._tokens = tokens
        self._tokenfreq = tokenfreq
        self._wordcount = wordcount
        self._revtokens = revtokens
        return self._tokens
    
    def sentences(self):
        if hasattr(self, "_sentences") and self._sentences:
            return self._sentences

        sentences = []
        with open(self.path + "/datasetSentences.txt", "r") as f:
            first = True
            for line in f:
                if first:
                    first = False
                    continue

                splitted = line.strip().split()[1:]
                # Deal with some peculiar encoding issues with this file
                sentences += [[w.lower().decode("utf-8").encode('latin1') for w in splitted]]
                
        self._sentences = sentences
        self._sentlengths = np.array([len(s) for s in sentences])
        self._cumsentlen = np.cumsum(self._sentlengths)

        return self._sentences

    def numSentences(self):
        if hasattr(self, "_numSentences") and self._numSentences:
            return self._numSentences
        else:
            self._numSentences = len(self.sentences())
            return self._numSentences

    def allSentences(self):
        if hasattr(self, "_allsentences") and self._allsentences:
            return self._allsentences

        sentences = self.sentences()
        rejectProb = self.rejectProb()
        tokens = self.tokens()
        allsentences = [[w for w in s 
            if 0 >= rejectProb[tokens[w]] or random.random() >= rejectProb[tokens[w]]]
            for s in sentences * 30]

        allsentences = [s for s in allsentences if len(s) > 1]
        
        self._allsentences = allsentences
        
        return self._allsentences

    def getRandomContext(self, C=5):
        allsent = self.allSentences()
        sentID = random.randint(0, len(allsent) - 1)
        sent = allsent[sentID]
        wordID = random.randint(0, len(sent) - 1)

        context = sent[max(0, wordID - C):wordID] 
        if wordID+1 < len(sent):
            context += sent[wordID+1:min(len(sent), wordID + C + 1)]

        centerword = sent[wordID]
        context = [w for w in context if w != centerword]

        if len(context) > 0:
            return centerword, context
        else:
            return self.getRandomContext(C)

    def sent_labels(self):
        if hasattr(self, "_sent_labels") and self._sent_labels:
            return self._sent_labels

        dictionary = dict()
        phrases = 0
        with open(self.path + "/dictionary.txt", "r") as f:
            for line in f:
                line = line.strip()
                if not line: continue
                splitted = line.split("|")
                dictionary[splitted[0].lower()] = int(splitted[1])
                phrases += 1

        labels = [0.0] * phrases
        with open(self.path + "/sentiment_labels.txt", "r") as f:
            first = True
            for line in f:
                if first:
                    first = False
                    continue

                line = line.strip()
                if not line: continue
                splitted = line.split("|")
                labels[int(splitted[0])] = float(splitted[1])

        sent_labels = [0.0] * self.numSentences()
        sentences = self.sentences()
        for i in xrange(self.numSentences()):
            sentence = sentences[i]
            full_sent = " ".join(sentence).replace('-lrb-', '(').replace('-rrb-', ')')
            sent_labels[i] = labels[dictionary[full_sent]]
            
        self._sent_labels = sent_labels
        return self._sent_labels

    def dataset_split(self):
        if hasattr(self, "_split") and self._split:
            return self._split

        split = [[] for i in xrange(3)]
        with open(self.path + "/datasetSplit.txt", "r") as f:
            first = True
            for line in f:
                if first:
                    first = False
                    continue

                splitted = line.strip().split(",")
                split[int(splitted[1]) - 1] += [int(splitted[0]) - 1]

        self._split = split
        return self._split

    def getRandomTrainSentence(self):
        split = self.dataset_split()
        sentId = split[0][random.randint(0, len(split[0]) - 1)]
        return self.sentences()[sentId], self.categorify(self.sent_labels()[sentId])

    def categorify(self, label):
        if label <= 0.2:
            return 0
        elif label <= 0.4:
            return 1
        elif label <= 0.6:
            return 2
        elif label <= 0.8:
            return 3
        else:
            return 4

    def getDevSentences(self):
        return self.getSplitSentences(2)

    def getTestSentences(self):
        return self.getSplitSentences(1)

    def getTrainSentences(self):
        return self.getSplitSentences(0)

    def getSplitSentences(self, split=0):
        ds_split = self.dataset_split()
        return [(self.sentences()[i], self.categorify(self.sent_labels()[i])) for i in ds_split[split]]

    def sampleTable(self):
        if hasattr(self, '_sampleTable') and self._sampleTable is not None:
            return self._sampleTable

        nTokens = len(self.tokens())
        samplingFreq = np.zeros((nTokens,))
        self.allSentences()
        i = 0
        for w in xrange(nTokens):
            w = self._revtokens[i]
            if w in self._tokenfreq:
                freq = 1.0 * self._tokenfreq[w]
                # Reweigh
                freq = freq ** 0.75
            else:
                freq = 0.0
            samplingFreq[i] = freq
            i += 1

        samplingFreq /= np.sum(samplingFreq)
        samplingFreq = np.cumsum(samplingFreq) * self.tablesize

        self._sampleTable = [0] * self.tablesize

        j = 0
        for i in xrange(self.tablesize):
            while i > samplingFreq[j]:
                j += 1
            self._sampleTable[i] = j

        return self._sampleTable

    def rejectProb(self):
        if hasattr(self, '_rejectProb') and self._rejectProb is not None:
            return self._rejectProb

        threshold = 1e-5 * self._wordcount

        nTokens = len(self.tokens())
        rejectProb = np.zeros((nTokens,))
        for i in xrange(nTokens):
            w = self._revtokens[i]
            freq = 1.0 * self._tokenfreq[w]
            # Reweigh
            rejectProb[i] = max(0, 1 - np.sqrt(threshold / freq))

        self._rejectProb = rejectProb
        return self._rejectProb

    def sampleTokenIdx(self):
        return self.sampleTable()[random.randint(0, self.tablesize - 1)]

## run word2vec

In [ ]:
import random
import numpy as np
# from cs224.data_utils import *    
import matplotlib.pyplot as plt

# from c5_word2vec import *
# from c6_sgd import *

random.seed(314)
dataset = StanfordSentiment()
tokens = dataset.tokens()
nWords = len(tokens)

# We are going to train 10-dimensional vectors for this assignment
dimVectors = 10

# Context size
C = 5

# Reset the random seed to make sure that everyone gets the same results
random.seed(31415)
np.random.seed(9265)

wordVectors = np.concatenate(((np.random.rand(nWords, dimVectors) - .5) / \
	dimVectors, np.zeros((nWords, dimVectors))), axis=0)
	
wordVectors0 = sgd(
	lambda vec: word2vec_sgd_wrapper(skipgram, tokens, vec, dataset, C,
		negSamplingCostAndGradient),
		wordVectors, 0.3, 40000, None, True, PRINT_EVERY=10)

print "sanity check: cost at convergence should be around or below 10"

wordVectors = (wordVectors0[:nWords, :] + wordVectors0[nWords:, :])

_, wordVectors0, _ = load_saved_params()

wordVectors = (wordVectors0[:nWords, :] + wordVectors0[nWords:, :])

visualizeWords = ["the", "a", "an", ",", ".", "?", "!", "``", "''", "--", 
	"good", "great", "cool", "brilliant", "wonderful", "well", "amazing",
	"worth", "sweet", "enjoyable", "boring", "bad", "waste", "dumb", 
	"annoying"]

visualizeIdx = [tokens[word] for word in visualizeWords]
visualizeVecs = wordVectors[visualizeIdx, :]
temp = (visualizeVecs - np.mean(visualizeVecs, axis=0))
covariance = 1.0 / len(visualizeIdx) * temp.T.dot(temp)
U, S, V = np.linalg.svd(covariance)
coord = temp.dot(U[:, 0:2])

for i in xrange(len(visualizeWords)):
	plt.text(coord[i, 0], coord[i, 1], visualizeWords[i],
		bbox=dict(facecolor='green', alpha=0.1))

plt.xlim((np.min(coord[:, 0]), np.max(coord[:, 0])))
plt.ylim((np.min(coord[:, 1]), np.max(coord[:, 1])))

plt.savefig('q3_word_vectors.png')
plt.show()
